In [2]:
# Import statements

import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [154]:
# Paths and Constants
top_words_size = 4000

word_spam_prob_file = 'prob-spam.txt'
word_ham_prob_file = 'prob-ham.txt'
word_all_prob_file ='prob-all.txt'

test_words_matrix = 'test-words.txt'
test_target_file = 'test-target.txt'
test_email_csv='test-email-csv.txt'

top_words = pd.read_csv('topwords.csv')
# Prior (Guess) in Bayesian Statistics 
prob_spam = 0.3113

In [94]:
# Load the Probabilities
prob_word_spam = np.loadtxt(word_spam_prob_file, delimiter=' ')
prob_word_ham = np.loadtxt(word_ham_prob_file, delimiter=' ')
prob_word_all = np.loadtxt(word_all_prob_file, delimiter=' ')


[8.49960417e-03 4.58874336e-03 6.42329261e-03 ... 4.26639362e-05
 4.74043735e-06 9.48087471e-06]


In [192]:
# Clean Email body function
stemmer = PorterStemmer()

def clean_email_body(email):
    """
    email : user given email to filter
    
    returns : a list of tokenized, filtered words from the input given to the function
    """
    filtered_words = []
    words_in_email = word_tokenize(email)
    for word in words_in_email: 
        if word.isalpha() and word not in stopwords.words('english'):
            filtered_words.append(stemmer.stem(word.lower()))
            
    return filtered_words

In [150]:
example_email = "Dear mom, I am out with my friends playing football. So, I will arrive home late. Please leave the door open."

example_email_1 = "Dear user, PLease click on the link below to get the prize the you have won"
example_3 = """Hi Andrew,

I plan on making a simpler form of naive Bayes classifier using conditional Bayes probability theorem that I learnt in my high school (Attached with this email). Following are some steps that I will be following : 

1. Gathering the Spam Assasin Corpus : https://spamassassin.apache.org/old/publiccorpus/
2. Preprocessing the spam and non spam emails by removing any html tags present in the email, word stemming and removing punctuations.
3. Putting all the email bodies in a pandas dataframe and tokenizing.
4. Finding occurrences of words in both spam and nonspam emails. (I plan on using the top 4000 most occuring words)
5. Finding the probability of occurrence of words in spam and nonspam emails (Conditional Probability).
6. Implementing Bayes Theorem to the result found from above ie. 5. 



Throughout my final project, I will be referencing Andrew Ng videos and the statsoft's bayes classifier overview. I will be evaluating the effectiveness using the idea of false and true positives, negatives. """

email_body = "Dear Suman, I am a little busy this morning and won't be in my office. Please meet me on monday during my office hours. I can help you with the final project and will also talk to you about your plan for winter. I will provide you with guidance."

In [190]:
def predict_user_email():
    """
    Function that checks if a user given email is spam or not. 
    This function asks the user for email as input and checks whether it is spam or not.
    
    Return : Spam for spam detection and Non-Spam for non spam detection
    
    """
    
    email = str(input("Enter an email: "))
    email = clean_email_body(email)
    
    
    # Empty df
    column_names = list(range(0, top_words_size))
    empty_df = pd.DataFrame(index=['User_Email'], columns=column_names)
    empty_df.fillna(value=0, inplace=True)
    # Populate the empty dataframe
    for word in email:
        if word in set(top_words.Top_Words):
            
            idx = top_words.Word_ID.loc[top_words.Top_Words == word].values[0]
            
            empty_df[idx] = empty_df[idx] + 1
            
    populated_df = empty_df
    
    populated_df.to_csv(test_email_csv)
    # Get the probability of the whole email being spam and not spam
    full_email_spam = populated_df.values.dot((np.log(prob_word_spam) - np.log(prob_word_all)))+ np.log(prob_spam)
    full_email_ham = populated_df.values.dot((np.log(prob_word_ham) - np.log(prob_word_all))) + np.log(1-prob_spam)
    # Check which probability is greater
    result = full_email_spam[0] > full_email_ham[0]
    
    if result:

        return "Spam"
    else:
        return "Non-Spam"



In [191]:
predict_user_email()

Enter an email: Hi Andrew, Can you meet me during my office hour today?


'Non-Spam'

In [189]:
top_words.iloc[2692]

Word_ID        2692
Top_Words    andrew
Name: 2692, dtype: object